# Welcome to the manual data parser

### The first step is to import the packages necessary to format and clean the data
#### If you're having trouble running the script, ensure you have the pandas packed installed by opening "windows powershell", and type, "pip install pandas" and hit enter

In [1]:
import pandas as pd
import numpy as np

### At the this time, the Enspire raw file and OD file need the file path manually copied and pasted - no formatting is necessary - just select copy path in folder location

In [2]:
enspire_path = r"L:\Assay Development\amber brown\SSF HPB runs\SSF00599\SSF00599_enspire.csv"

In [3]:
od_path = r"L:\Molecular Sciences\Small Scale Runs\SSF00599 HTP v2 (QC) Assess 96DW plate alternatives_MS\Assays\OD\Processed\SSF00599 HTP OD600_copy.xlsx"

### These are the well ids that will link the Enspire values back to the source value, as well as link the Enspire data to the OD data

In [4]:
well_ids = ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10',
             'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10',
             'C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10', 
             'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10',
             'E01', 'E02', 'E03', 'E04', 'E05', 'E06', 'E07', 'E08', 'E09', 'E10',
             'F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'F10',
             'G01', 'G02', 'G03', 'G04', 'G05', 'G06', 'G07', 'G08', 'G09', 'G10',
             'H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10',
             ]

### This is a function that will be called later in the script. Once the data is cleaned and formatted, this function will make calculations
##### Note that the "dilution_volumes = " list values will need to be replace with experimental values!

In [5]:
def calculations(clean_df, standard=0):
    
    ### Replace the values within the brackets to the experimental volumes used! 
    ### Enter as integer or float separated by comma within the square brackets
    dilution_volumes = [1, 0.166666667, 0.027777778, 0.00462963, 0.000771605, 0.000128601]
    
    if standard == 1:
        clean_df[["STD_1", "STD_2", "STD_3", "STD_4", "STD_5", "STD_6"]] = clean_df[
            ["STD_1", "STD_2", "STD_3", "STD_4", "STD_5", "STD_6"]].astype(float)
        for n in range(1, 6):
            clean_df[f"std_slope_{n}"] = round((clean_df[f"STD_{n + 1}"] - clean_df[f"STD_{n}"]) /
                                              (dilution_volumes[n] - dilution_volumes[n -1]), 2)

        clean_df["max_std_slope"] = clean_df[
            ["std_slope_1", "std_slope_2", "std_slope_3", "std_slope_4", "std_slope_5"]
        ].max(axis=1)
        
        return clean_df
    
    else:
        
        clean_df[["Alpha_1", "Alpha_2", "Alpha_3", "Alpha_4", "Alpha_5", "Alpha_6"]] = clean_df[
            ["Alpha_1", "Alpha_2", "Alpha_3", "Alpha_4", "Alpha_5", "Alpha_6"]
        ].astype(float)
        
        clean_df[["DNA_1", "DNA_2", "DNA_3", "DNA_4", "DNA_5", "DNA_6"]] = clean_df[
            ["DNA_1", "DNA_2", "DNA_3", "DNA_4", "DNA_5", "DNA_6"]
        ].astype(float)

        for n in range(1, 6):
            clean_df[f"alpha_slope_{n}"] = round((clean_df[f"Alpha_{n + 1}"] - (clean_df[f"Alpha_{n}"])) /
                                                 (dilution_volumes[n] - dilution_volumes[n - 1]), 2)
        clean_df["max_alpha_slope"] = clean_df[
            ["alpha_slope_1", "alpha_slope_2", "alpha_slope_3", "alpha_slope_4", "alpha_slope_5"]
        ].max(axis=1)

        for n in range(1, 6):
            clean_df[f"dna_slope_{n}"] = round((clean_df[f"DNA_{n + 1}"] - clean_df[f"DNA_{n}"]) /
                                                    (dilution_volumes[n] - dilution_volumes[n - 1]), 2)
        clean_df["max_dna_slope"] = clean_df[
            ["dna_slope_1", "dna_slope_2", "dna_slope_3", "dna_slope_4", "dna_slope_5"]
        ].max(axis=1)

        clean_df["max_alpha_slope/max_dna_slope"] = round(clean_df["max_alpha_slope"] / clean_df["max_dna_slope"], 2)

        return clean_df

### This section of code will import the OD file, format the column headings, remove empty data, and drop unnecessary columns, and create a unique ID that will help link the data to the Enspire data from the dataframe

In [6]:
od_data = pd.read_excel(od_path)

In [7]:
for column in od_data.columns:
    od_data.rename(columns={column: column.title()}, inplace=True)
od_data.dropna(subset=["Ssf Exp"], inplace=True)
od_data["Od600"] = od_data["Od600"].replace(" ", "0.0")

In [8]:
drop_cols = ["Ssf Exp", "Seed Plate", "Seed Well", "Harvest Plate", "Harvest Well"]
od_data_clean = od_data.drop(columns=drop_cols)


In [1]:
od_data_clean.head()

NameError: name 'od_data_clean' is not defined

In [10]:
od_data_clean.insert(0, "Id", od_data_clean["Harvest Sample Id"].apply(lambda x: str(x).split("-", 1)[-1]))
od_data_clean.insert(1, "Plate", od_data_clean["Id"].apply(lambda x:str(x).split("-")[0]))
od_data_clean.insert(2, "Well_Id", od_data_clean["Id"].apply(lambda x:str(x).split("-")[1]))

### This code will show a snapshot of the newly formatted datafframe

In [11]:
od_data_clean.head()

,Id,Plate,Well_Id,Row,Column,Seed Sample Id,Harvest Sample Id,Sample Type,Strain,Bio-Replicate No.,Induction Temp,Proprionate (Mm),Arabinose (Um),Od600
0,P1-A01,P1,A01,A,3.0,SSF00599-P1-A03,SSF00599-P1-A01,negative control,ABS08019,1.0,21 C,20.0,250.0,15.078720
1,P1-A02,P1,A02,A,9.0,SSF00599-P1-A09,SSF00599-P1-A02,positive control,ABS11963,1.0,21 C,20.0,250.0,3.093408
2,P1-A03,P1,A03,A,3.0,SSF00599-P2-A03,SSF00599-P1-A03,negative control,ABS08019,1.0,21 C,20.0,250.0,16.090825
3,P1-A04,P1,A04,A,9.0,SSF00599-P2-A09,SSF00599-P1-A04,positive control,ABS11963,1.0,21 C,20.0,250.0,2.692000
4,P1-A05,P1,A05,A,3.0,SSF00599-P3-A03,SSF00599-P1-A05,negative control,ABS08019,1.0,21 C,20.0,250.0,10.496483


### The next bloc of code will import the Enspire data, creating a dataframe for the alpha signals and dna signals respectively. 
##### Note that this data will require manual input of plate ids and source plates at the " source_plates = [] " and " plates  = [] " variables - please enter in the following format: ["P1", "P2", "P3"] 

In [12]:
count = 0
source_plates = ["P1"]
plates = ["P1", "P2", "P3", "P4"]
all_alpha_df = pd.DataFrame()
all_dna_df = pd.DataFrame()

In [13]:
for plate in plates:
    alpha_df = pd.read_csv(
        enspire_path,
        header=None,
        names=np.arange(0,25),
        usecols=np.arange(0, 25), 
        skiprows=7 + count * 48, 
        nrows=16
    )
    alpha_df.insert(1, "ID", f"{plate}" + "-" + alpha_df[0])
    alpha_df.drop(0, axis=1, inplace=True)
    all_alpha_df = pd.concat([all_alpha_df, alpha_df])
    
    dna_df = pd.read_csv(
        enspire_path,
        header=None,
        names=np.arange(24, 49),
        usecols=np.arange(0, 25),
        skiprows=31 + count * 48,
        nrows=16
    )
    dna_df.insert(1, "ID", f"{plate}" + "-" + dna_df[24])
    dna_df.drop(24, axis=1, inplace=True)
    all_dna_df = pd.concat([all_dna_df, dna_df])

    count += 1

### This next bloc of code will prepare empty dataframes where data from the enspire file will be captured and concatenated. 
##### Note that the "std_conc = " list values will need to be replaced with concentrations used in experiment! Please enter values as integer or float separated by a comma, inside of square brackets '[ ]' 

In [14]:
alpha_sig_cols = ["ProxiPlate"] + [f"Alpha_{n}" for n in range(1, 7)]
all_alpha_concat_df = pd.DataFrame(columns=alpha_sig_cols)
all_rep_concat_df = pd.DataFrame(columns=alpha_sig_cols)

dna_sig_cols = ["ProxiPlate"] + [f"DNA_{n}" for n in range(1, 7)]
all_dna_concat_df = pd.DataFrame(columns=dna_sig_cols)
all_dna_rep_concat_df = pd.DataFrame(columns=dna_sig_cols)

std_sig_cols = ["ProxiPlate"] + ["Concentration (nm)"] + [f"STD_{n}" for n in range(1, 7)]
all_std_concat_df = pd.DataFrame(columns=std_sig_cols)

std_section = "A B C D".split()

# Replace values with experimental values!
std_conc = [50, 16.7, 5.6, 1.9]

# Row and column counters the will help capture the data we need for our end dataframe
s_row = 0
e_row = 10
std_s_row = 12
std_e_row = 16

# This indicates the section of the plate the for loop will loop through
sections = [1, 2, 3, 4]

### This next bloc creates a nested for loop that will loop through each plate, and each section of the plate to capture the data we want, and concatenate data to specified dataframe, separating Alpha signals, DNA signals, and standards

In [15]:
for plate in plates:
    s_col = 1
    e_col = 7
    for section in sections:
        alpha_capture_df = all_alpha_df.iloc[s_row: e_row, s_col: e_col]
        alpha_capture_df.insert(0, "ProxiPlate", plate)
        alpha_capture_df.columns= alpha_sig_cols

        dna_capture_df = all_dna_df.iloc[s_row: e_row, s_col: e_col]
        dna_capture_df.insert(0, "ProxiPlate", plate)
        dna_capture_df.columns= dna_sig_cols

        std_capture_df = all_alpha_df.iloc[std_s_row:std_e_row, s_col: e_col]
        std_capture_df.insert(0, "ProxiPlate", f"{plate}-{std_section[section - 1]}")
        std_capture_df.insert(1, "Concentration (nm)", std_conc)
        std_capture_df.columns= std_sig_cols

        if section % 2 != 0:
            all_alpha_concat_df = pd.concat([all_alpha_concat_df, alpha_capture_df])
            all_dna_concat_df = pd.concat([all_dna_concat_df, dna_capture_df])
            all_std_concat_df = pd.concat([all_std_concat_df, std_capture_df])

        else:
            all_rep_concat_df = pd.concat([all_rep_concat_df, alpha_capture_df])
            all_dna_rep_concat_df = pd.concat([all_dna_rep_concat_df, dna_capture_df])
            all_std_concat_df = pd.concat([all_std_concat_df, std_capture_df])

        s_col += 6
        e_col += 6
        
    s_row += 16
    e_row += 16
    std_s_row += 16
    std_e_row += 16

### Here the unique ID will be created, which will be used to merge Alpha and DNA signals to one dataframe, and eventuallly used to link to OD dataframe

In [16]:
all_alpha_concat_df.insert(1, "Well_Id", well_ids)
all_alpha_concat_df.insert(2, "HPB_Id", all_alpha_concat_df["ProxiPlate"] + "-" + all_alpha_concat_df["Well_Id"])
all_alpha_concat_df.insert(0, "Id", source_plates[0] + "-" + all_alpha_concat_df["Well_Id"])

all_rep_concat_df.insert(1, "Well_Id", well_ids)
all_rep_concat_df.insert(2, "HPB_Id", all_rep_concat_df["ProxiPlate"] + "-" + all_rep_concat_df["Well_Id"])
all_rep_concat_df.insert(0, "Id", source_plates[0] + "-" + all_rep_concat_df["Well_Id"])

all_dna_concat_df.insert(1, "Well_Id", well_ids)
all_dna_concat_df.insert(2, "HPB_Id", all_dna_concat_df["ProxiPlate"] + "-" + all_dna_concat_df["Well_Id"])
all_dna_concat_df.insert(0, "Id", source_plates[0] + "-" + all_dna_concat_df["Well_Id"])

all_dna_rep_concat_df.insert(1, "Well_Id", well_ids)
all_dna_rep_concat_df.insert(2, "HPB_Id", all_dna_rep_concat_df["ProxiPlate"] + "-" + all_dna_rep_concat_df["Well_Id"])
all_dna_rep_concat_df.insert(0, "Id", source_plates[0] + "-" + all_dna_rep_concat_df["Well_Id"])

### These two lines will merge Alpha and DNA signals for the main and replicate data respectively

In [17]:
main_df = pd.merge(all_alpha_concat_df, all_dna_concat_df, on=["Id", "ProxiPlate", "Well_Id", "HPB_Id"])
rep_df = pd.merge(all_rep_concat_df, all_dna_rep_concat_df, on=["Id", "ProxiPlate", "Well_Id","HPB_Id"])

### This is where the aforementioned function, "def calculations" is called. This will take each of the created data frames, and make the necessary slope calculations

In [18]:
calculations(main_df)
calculations(rep_df)
calculations(all_std_concat_df, standard=1)

,ProxiPlate,Concentration (nm),STD_1,STD_2,STD_3,STD_4,STD_5,STD_6,std_slope_1,std_slope_2,std_slope_3,std_slope_4,std_slope_5,max_std_slope
12,P1-A,50.0,62584.0,52195.0,2773.0,1011.0,762.0,971.0,12466.8,355838.4,76118.4,64540.79,-325036.86,355838.4
13,P1-A,16.7,62820.0,6389.0,951.0,868.0,957.0,1397.0,67717.2,39153.6,3585.6,-23068.80,-684288.12,67717.2
14,P1-A,5.6,25683.0,1249.0,990.0,917.0,903.0,2923.0,29320.8,1864.8,3153.6,3628.80,-3141504.56,29320.8
15,P1-A,1.9,2818.0,1310.0,866.0,1151.0,969.0,2030.0,1809.6,3196.8,-12312.0,47174.40,-1650067.50,47174.4
12,P1-B,50.0,54960.0,43822.0,3587.0,2322.0,2400.0,2244.0,13365.6,289692.0,54648.0,-20217.60,242611.24,289692.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,P4-C,1.9,3392.0,1362.0,1185.0,1138.0,1152.0,1192.0,2436.0,1274.4,2030.4,-3628.80,-62208.01,2436.0
12,P4-D,50.0,84112.0,61885.0,4195.0,1382.0,1516.0,1408.0,26672.4,415368.0,121521.6,-34732.80,167961.63,415368.0
13,P4-D,16.7,95279.0,9288.0,1440.0,1182.0,1172.0,1224.0,103189.2,56505.6,11145.6,2592.00,-80870.41,103189.2
14,P4-D,5.6,32766.0,1916.0,1271.0,1190.0,1274.0,1311.0,37020.0,4644.0,3499.2,-21772.80,-57542.41,37020.0


In [19]:
main_df.shape

(80, 29)

### Now, the main and replicate dataframe will be merged with the OD dataframe based on the unique ID provided

In [20]:
main_and_od = pd.merge(od_data_clean, main_df, on=["Id", "Well_Id"])
rep_and_od = pd.merge(od_data_clean, rep_df, on=["Id", "Well_Id"])

### Last, our data will be exported to an excel spreadsheet, with the main data, replicate data, and standard data going each to a separate sheet within the workbook

In [21]:
with pd.ExcelWriter("Manual_Data.xlsx") as writer:
    main_and_od.to_excel(writer, sheet_name="Source_Signals")
    rep_and_od.to_excel(writer, sheet_name="Rep_Signals")
    all_std_concat_df.to_excel(writer, sheet_name="Standards")
